# 🎯 프롬프트 최적화 노트북

이 노트북은 프롬프트를 인터랙티브하게 편집하고 테스트하여 최적의 프롬프트를 찾을 수 있도록 도와줍니다.

## 주요 기능
- 📝 **인터랙티브 프롬프트 편집**: 셀에서 직접 프롬프트 수정 후 즉시 테스트
- 💾 **결과 자동 저장**: 타임스탬프 기반 파일 저장 (`test_results/`)
- 📊 **결과 비교**: 저장된 결과들을 나란히 비교
- 🔄 **파이프라인별 테스트**: info_retrieval과 script_gen 독립 실행

## 사용 방법
1. 원하는 파이프라인의 섹션으로 이동
2. 프롬프트를 직접 수정하거나 기존 버전 선택
3. 테스트 실행
4. 결과 저장
5. 여러 프롬프트 결과 비교

## ⚙️ 초기 설정

In [1]:
# 필요한 라이브러리 임포트
import sys
import os
from pathlib import Path
from datetime import datetime
import json
import yaml
from IPython.display import display, Markdown, HTML
import difflib

# 프로젝트 루트를 찾기 (더 robust한 방법)
# prompts 디렉토리가 있는 위치를 프로젝트 루트로 판단
current = Path.cwd()
if (current / 'prompts').exists():
    project_root = current
    print("📁 현재 디렉토리가 프로젝트 루트입니다")
elif (current.parent / 'prompts').exists():
    project_root = current.parent
    print("📁 상위 디렉토리가 프로젝트 루트입니다")
else:
    # 수동으로 지정
    print("⚠️ 자동으로 프로젝트 루트를 찾을 수 없습니다.")
    project_root = Path('/Users/jacti/develop/script_gen_v2')  # 절대 경로로 fallback
    print(f"📁 수동으로 설정된 프로젝트 루트: {project_root}")

# sys.path에 추가
sys.path.insert(0, str(project_root))

# 작업 디렉토리를 프로젝트 루트로 변경 (상대 경로가 제대로 작동하도록)
os.chdir(project_root)

# 파이프라인 모듈 임포트
from src.pipelines import info_retrieval, script_gen
from src.utils.prompt_loader import load_prompt, list_prompts
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv()

print("✅ 초기 설정 완료")
print(f"📁 프로젝트 루트: {project_root}")
print(f"📂 현재 작업 디렉토리: {Path.cwd()}")

# 프롬프트 디렉토리 확인
if (Path.cwd() / 'prompts').exists():
    print("✅ prompts 디렉토리를 찾았습니다")
else:
    print("❌ prompts 디렉토리를 찾을 수 없습니다. 경로를 확인하세요.")

📁 상위 디렉토리가 프로젝트 루트입니다
✅ 초기 설정 완료
📁 프로젝트 루트: /Users/jacti/develop/script_gen_v2
📂 현재 작업 디렉토리: /Users/jacti/develop/script_gen_v2
✅ prompts 디렉토리를 찾았습니다


In [2]:
# 결과 저장/비교를 위한 헬퍼 함수

def save_result(pipeline_type, prompt_version, keyword, result_text, metadata=None):
    """
    테스트 결과를 타임스탬프 기반 파일로 저장
    
    Args:
        pipeline_type: 'info_retrieval' 또는 'script_generation'
        prompt_version: 프롬프트 버전 이름
        keyword: 테스트에 사용한 키워드
        result_text: 생성된 결과 텍스트
        metadata: 추가 메타데이터 (dict)
    
    Returns:
        저장된 파일 경로
    """
    # 타임스탬프 생성
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 파일명 생성 (안전한 문자만 사용)
    safe_keyword = "".join(c if c.isalnum() or c in " _-" else "_" for c in keyword)
    filename = f"{timestamp}_{prompt_version}_{safe_keyword}.md"
    
    # 저장 경로
    save_dir = project_root / "test_results" / pipeline_type
    save_dir.mkdir(parents=True, exist_ok=True)
    save_path = save_dir / filename
    
    # 메타데이터 헤더 생성
    meta_info = {
        "pipeline": pipeline_type,
        "prompt_version": prompt_version,
        "keyword": keyword,
        "timestamp": timestamp,
        **(metadata or {})
    }
    
    header = "---\n"
    header += yaml.dump(meta_info, allow_unicode=True)
    header += "---\n\n"
    
    # 파일 저장
    with open(save_path, 'w', encoding='utf-8') as f:
        f.write(header)
        f.write(result_text)
    
    print(f"💾 결과 저장 완료: {save_path}")
    return save_path


def list_saved_results(pipeline_type):
    """
    저장된 결과 파일 목록 조회
    
    Args:
        pipeline_type: 'info_retrieval' 또는 'script_generation'
    
    Returns:
        파일 경로 리스트 (최신순)
    """
    results_dir = project_root / "test_results" / pipeline_type
    if not results_dir.exists():
        return []
    
    files = sorted(results_dir.glob("*.md"), key=lambda p: p.stat().st_mtime, reverse=True)
    return files


def compare_results(file1_path, file2_path):
    """
    두 결과 파일을 비교하여 diff 출력
    
    Args:
        file1_path: 첫 번째 파일 경로
        file2_path: 두 번째 파일 경로
    """
    with open(file1_path, 'r', encoding='utf-8') as f:
        content1 = f.read()
    
    with open(file2_path, 'r', encoding='utf-8') as f:
        content2 = f.read()
    
    # 메타데이터 제거 (비교용)
    if content1.startswith("---"):
        content1 = content1.split("---", 2)[2].strip()
    if content2.startswith("---"):
        content2 = content2.split("---", 2)[2].strip()
    
    # Diff 생성
    diff = difflib.unified_diff(
        content1.splitlines(keepends=True),
        content2.splitlines(keepends=True),
        fromfile=str(file1_path.name),
        tofile=str(file2_path.name),
        lineterm=''
    )
    
    print("\n📊 결과 비교:")
    print("=" * 80)
    print(f"파일 1: {file1_path.name}")
    print(f"파일 2: {file2_path.name}")
    print("=" * 80)
    
    diff_text = ''.join(diff)
    if diff_text:
        print(diff_text)
    else:
        print("✅ 두 파일의 내용이 동일합니다.")

print("✅ 헬퍼 함수 정의 완료")

✅ 헬퍼 함수 정의 완료


## 🔍 Pipeline 1: 정보 검색 (Info Retrieval) 프롬프트 테스트

In [3]:
# 사용 가능한 프롬프트 버전 확인
print("📋 사용 가능한 정보 검색 프롬프트 버전:\n")

info_prompts = list_prompts(pipeline_type="info_retrieval")
for version in info_prompts:
    try:
        template = load_prompt(version, pipeline_type="info_retrieval")
        print(f"📝 {version}:")
        print(f"   이름: {template.name}")
        print(f"   설명: {template.description}")
        print(f"   API: {template.api_type}")
        print()
    except Exception as e:
        print(f"❌ {version}: 로드 실패 - {e}\n")

2025-11-12 09:13:24,509 - src.utils.prompt_loader - INFO - 프롬프트 템플릿 로드: prompts/info_retrieval/default.yaml


📋 사용 가능한 정보 검색 프롬프트 버전:

📝 default:
   이름: 기본 정보 검색
   설명: 문화유산에 대한 구조화된 정보 검색 (개요, 역사, 특징, 추가 정보, 참고 자료)
   API: responses



In [4]:
# 정보 검색 프롬프트 테스트

# === 설정 ===
INFO_KEYWORD = "백제의 대표 유물 금동대향로의 제작 기술과 예술적 가치, 그리고 향 문화의 의미를 중심으로 설명해주세요."  # 테스트할 키워드
INFO_PROMPT_VERSION = "default"  # 사용할 프롬프트 버전
INFO_MODEL = "gpt-4.1"  # 사용할 모델
DRY_RUN = False  # True로 설정하면 API 호출 없이 목업 데이터 사용

# ⚠️ 중요: 동일한 keyword로 Pipeline 2도 실행할 예정이라면 output_name을 생략하세요!
# 또는 Pipeline 2에서도 동일한 output_name을 사용하세요.
USE_OUTPUT_NAME = False  # True로 설정하면 커스텀 output_name 사용
CUSTOM_OUTPUT_NAME = "금동대향로"  # output_name 사용 시 여기 지정

# === 실행 ===
print(f"🔍 정보 검색 시작...")
print(f"   키워드: {INFO_KEYWORD}")
print(f"   프롬프트 버전: {INFO_PROMPT_VERSION}")
print(f"   모델: {INFO_MODEL}")
print(f"   Dry-run: {DRY_RUN}")
if USE_OUTPUT_NAME:
    print(f"   Output name: {CUSTOM_OUTPUT_NAME}")
print()

try:
    # output_name 파라미터 준비
    kwargs = {
        "keyword": INFO_KEYWORD,
        "model": INFO_MODEL,
        "prompt_version": INFO_PROMPT_VERSION,
        "dry_run": DRY_RUN,
    }
    
    if USE_OUTPUT_NAME:
        kwargs["output_name"] = CUSTOM_OUTPUT_NAME
    
    output_path = info_retrieval.run(**kwargs)
    
    # 결과 읽기
    with open(output_path, 'r', encoding='utf-8') as f:
        result_text = f.read()
    
    print("✅ 정보 검색 완료!\n")
    print("=" * 80)
    print("📄 결과 미리보기:")
    print("=" * 80)
    print(result_text[:500] + "..." if len(result_text) > 500 else result_text)
    print("=" * 80)
    
    # 결과 저장 (선택적)
    save_choice = input("\n💾 이 결과를 test_results에 저장하시겠습니까? (y/n): ")
    if save_choice.lower() == 'y':
        saved_path = save_result(
            pipeline_type="info_retrieval",
            prompt_version=INFO_PROMPT_VERSION,
            keyword=INFO_KEYWORD,
            result_text=result_text,
            metadata={"model": INFO_MODEL, "dry_run": DRY_RUN}
        )
        print(f"✅ 저장 완료: {saved_path.name}")
    
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    import traceback
    traceback.print_exc()

2025-11-12 09:15:36,611 - src.pipelines.info_retrieval - INFO - 정보 검색 파이프라인 시작: 백제의 대표 유물 금동대향로의 제작 기술과 예술적 가치, 그리고 향 문화의 의미를 중심으로 설명해주세요.
2025-11-12 09:15:36,613 - src.pipelines.info_retrieval - INFO - 프롬프트 버전: default
2025-11-12 09:15:36,617 - src.pipelines.info_retrieval - INFO - 출력 디렉토리: /Users/jacti/develop/script_gen_v2/outputs/info
2025-11-12 09:15:36,701 - src.utils.prompt_loader - INFO - 프롬프트 템플릿 로드: prompts/info_retrieval/default.yaml
2025-11-12 09:15:36,703 - src.pipelines.info_retrieval - INFO - 프롬프트 템플릿 로드 완료: 기본 정보 검색 (버전: default)
2025-11-12 09:15:36,704 - src.pipelines.info_retrieval - INFO - 프롬프트 설명: 문화유산에 대한 구조화된 정보 검색 (개요, 역사, 특징, 추가 정보, 참고 자료)
2025-11-12 09:15:36,704 - src.pipelines.info_retrieval - INFO - API 타입: responses
2025-11-12 09:15:36,704 - src.pipelines.info_retrieval - INFO - LLM 검색 시작: 백제의 대표 유물 금동대향로의 제작 기술과 예술적 가치, 그리고 향 문화의 의미를 중심으로 설명해주세요. (모델: gpt-4.1)


🔍 정보 검색 시작...
   키워드: 백제의 대표 유물 금동대향로의 제작 기술과 예술적 가치, 그리고 향 문화의 의미를 중심으로 설명해주세요.
   프롬프트 버전: default
   모델: gpt-4.1
   Dry-run: False



2025-11-12 09:16:04,967 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-11-12 09:16:05,087 - src.pipelines.info_retrieval - INFO - LLM 검색 완료: 1640 글자
2025-11-12 09:16:05,089 - src.pipelines.info_retrieval - INFO - 파일 저장 완료: /Users/jacti/develop/script_gen_v2/outputs/info/백제의 대표 유물 금동대향로의 제작 기술과 예술적 가치, 그리고 향 문화의 의미를 중심으로 설명해주세요..md
2025-11-12 09:16:05,090 - src.utils.metadata - INFO - 메타데이터 저장 완료: outputs/info/백제의 대표 유물 금동대향로의 제작 기술과 예술적 가치, 그리고 향 문화의 의미를 중심으로 설명해주세요..md.metadata.json


✅ 정보 검색 완료!

📄 결과 미리보기:
# 백제 금동대향로

## 개요
백제 금동대향로(百濟金銅大香爐)는 1993년 충청남도 부여군 능산리 절터에서 출토된 백제시대의 대표적 걸작 유물입니다. 이 향로는 백제의 뛰어난 금속공예 기술과 불교적 미의식, 그리고 향 문화를 집대성한 유물로 평가받고 있습니다.

## 역사 및 배경
- **시대적 배경**: 삼국시대 백제, 약 6세기경(성왕~위덕왕 시기)
- **제작 시기 및 장소**: 6세기 중반, 웅진(현재의 공주) 또는 사비(현재의 부여) 일대에서 제작된 것으로 추정
- **역사적 의미**: 백제 금동대향로는 한반도에서 발견된 가장 화려하고 정교한 금동공예품 중 하나로, 백제의 기술과 예술의 정수를 보여줍니다. 불교가 번성하던 백제에서 의식용과 장식용, 권력의 상징적 용도로 사용된 것으로 추정됩니다.

## 주요 특징
- **외형적 특징**: 향로는 뚜껑, 몸통, 받침 세 부분으로 구성되어 있습니다. 전체 높이는 약 61.8cm, 무게 약 11.8kg입니다. 뚜껑에는 신선과 동...


## 📝 Pipeline 2: 스크립트 생성 (Script Generation) 프롬프트 테스트

In [5]:
# 사용 가능한 프롬프트 버전 확인
print("📋 사용 가능한 스크립트 생성 프롬프트 버전:\n")

script_prompts = list_prompts(pipeline_type="script_generation")
for version in script_prompts:
    try:
        template = load_prompt(version, pipeline_type="script_generation")
        print(f"📝 {version}:")
        print(f"   이름: {template.name}")
        print(f"   설명: {template.description}")
        print(f"   API: {template.api_type}")
        print()
    except Exception as e:
        print(f"❌ {version}: 로드 실패 - {e}\n")

2025-11-12 09:16:18,834 - src.utils.prompt_loader - INFO - 프롬프트 템플릿 로드: prompts/script_generation/v1.yaml
2025-11-12 09:16:18,839 - src.utils.prompt_loader - INFO - 프롬프트 템플릿 로드: prompts/script_generation/v2.yaml
2025-11-12 09:16:18,841 - src.utils.prompt_loader - INFO - 프롬프트 템플릿 로드: prompts/script_generation/v2-tts.yaml
2025-11-12 09:16:18,844 - src.utils.prompt_loader - INFO - 프롬프트 템플릿 로드: prompts/script_generation/v3-tts.yaml


📋 사용 가능한 스크립트 생성 프롬프트 버전:

📝 v1:
   이름: 기본 오디오 가이드
   설명: 친절하고 시각적 이미지를 유도하는 1분 내외 스크립트
   API: chat

📝 v2:
   이름: 교육적 오디오 가이드
   설명: 학습자를 위한 정보 중심적이고 체계적인 스크립트
   API: chat

📝 v2-tts:
   이름: 순수 스크립트 (TTS 최적화)
   설명: Markdown 헤더 없이 TTS가 바로 읽을 수 있는 순수 텍스트 스크립트
   API: chat

📝 v3-tts:
   이름: 순수 스크립트 (TTS 최적화)
   설명: Markdown 헤더 없이 TTS가 바로 읽을 수 있는 순수 텍스트 스크립트
   API: chat



In [6]:
# 스크립트 생성 프롬프트 테스트

# === 설정 ===
SCRIPT_KEYWORD = "훈민정음"  # 테스트할 키워드 (Pipeline 1에서 생성한 정보 파일 필요)
SCRIPT_PROMPT_VERSION = "v3-tts"  # 사용할 프롬프트 버전
SCRIPT_MODEL = "gpt-4.1"  # 사용할 모델
TEMPERATURE = 0.7  # 창의성 조절 (0.0~1.0)
DRY_RUN = False  # True로 설정하면 API 호출 없이 목업 데이터 사용

# ⚠️ 중요: output_name은 Pipeline 1과 동일하게 유지하거나 생략하세요!
# Pipeline 1에서 output_name을 사용했다면 여기서도 동일한 이름을 사용해야 합니다.
# 간단하게 하려면 output_name을 생략하고 keyword만 사용하세요.
USE_OUTPUT_NAME = False  # True로 설정하면 커스텀 output_name 사용
CUSTOM_OUTPUT_NAME = "test_prompt_comparison"  # output_name 사용 시 여기 지정

# === 실행 ===
print(f"📝 스크립트 생성 시작...")
print(f"   키워드: {SCRIPT_KEYWORD}")
print(f"   프롬프트 버전: {SCRIPT_PROMPT_VERSION}")
print(f"   모델: {SCRIPT_MODEL}")
print(f"   Temperature: {TEMPERATURE}")
print(f"   Dry-run: {DRY_RUN}")
if USE_OUTPUT_NAME:
    print(f"   Output name: {CUSTOM_OUTPUT_NAME}")
print()

try:
    # output_name 파라미터 준비
    kwargs = {
        "keyword": SCRIPT_KEYWORD,
        "prompt_version": SCRIPT_PROMPT_VERSION,
        "model": SCRIPT_MODEL,
        "temperature": TEMPERATURE,
        "dry_run": DRY_RUN,
    }
    
    if USE_OUTPUT_NAME:
        kwargs["output_name"] = CUSTOM_OUTPUT_NAME
        print(f"⚠️ 주의: Pipeline 1에서도 동일한 output_name '{CUSTOM_OUTPUT_NAME}'을 사용했는지 확인하세요!")
    
    output_path = script_gen.run(**kwargs)
    
    # 결과 읽기
    with open(output_path, 'r', encoding='utf-8') as f:
        result_text = f.read()
    
    print("✅ 스크립트 생성 완료!\n")
    print("=" * 80)
    print("📄 결과 미리보기:")
    print("=" * 80)
    print(result_text[:500] + "..." if len(result_text) > 500 else result_text)
    print("=" * 80)
    
    # 결과 저장 (선택적)
    save_choice = input("\n💾 이 결과를 test_results에 저장하시겠습니까? (y/n): ")
    if save_choice.lower() == 'y':
        saved_path = save_result(
            pipeline_type="script_generation",
            prompt_version=SCRIPT_PROMPT_VERSION,
            keyword=SCRIPT_KEYWORD,
            result_text=result_text,
            metadata={"model": SCRIPT_MODEL, "temperature": TEMPERATURE, "dry_run": DRY_RUN}
        )
        print(f"✅ 저장 완료: {saved_path.name}")
    
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    print("\n💡 해결 방법:")
    print("1. Pipeline 1 (정보 검색)을 먼저 실행하여 정보 파일을 생성하세요")
    print(f"   → 키워드: '{SCRIPT_KEYWORD}'")
    if USE_OUTPUT_NAME:
        print(f"   → output_name: '{CUSTOM_OUTPUT_NAME}' (동일하게)")
    print("2. 또는 DRY_RUN = True로 설정하여 테스트해보세요")
    print()
    import traceback
    traceback.print_exc()

2025-11-12 09:49:59,870 - src.pipelines.script_gen - INFO - 스크립트 생성 파이프라인 시작: 훈민정음
2025-11-12 09:49:59,871 - src.pipelines.script_gen - INFO - 프롬프트 버전: v3-tts
2025-11-12 09:49:59,872 - src.pipelines.script_gen - INFO - 입력 파일: outputs/info/훈민정음.md
2025-11-12 09:49:59,872 - src.pipelines.script_gen - INFO - 출력 파일: outputs/script/훈민정음_script.md
2025-11-12 09:49:59,872 - src.utils.prompt_loader - INFO - 프롬프트 템플릿 로드: prompts/script_generation/v3-tts.yaml
2025-11-12 09:49:59,874 - src.pipelines.script_gen - INFO - 프롬프트 템플릿 로드 완료: 순수 스크립트 (TTS 최적화)
2025-11-12 09:49:59,875 - src.pipelines.script_gen - INFO - 프롬프트 설명: Markdown 헤더 없이 TTS가 바로 읽을 수 있는 순수 텍스트 스크립트
2025-11-12 09:49:59,875 - src.pipelines.script_gen - INFO - 프롬프트 태그: 
2025-11-12 09:49:59,876 - src.pipelines.script_gen - INFO - 정보 파일 읽기 중...
2025-11-12 09:49:59,879 - src.pipelines.script_gen - INFO - 정보 파일 로드 완료 (길이: 1338 문자)
2025-11-12 09:49:59,882 - src.pipelines.script_gen - INFO - LLM 호출 시작 (모델: gpt-4.1, temperature: 0.7)


📝 스크립트 생성 시작...
   키워드: 훈민정음
   프롬프트 버전: v3-tts
   모델: gpt-4.1
   Temperature: 0.7
   Dry-run: False



2025-11-12 09:50:18,210 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-12 09:50:18,244 - src.pipelines.script_gen - INFO - LLM 응답 수신 완료 (길이: 1355 문자)
2025-11-12 09:50:18,247 - src.pipelines.script_gen - INFO - ✅ 스크립트 파일 저장 완료: outputs/script/훈민정음_script.md
2025-11-12 09:50:18,248 - src.utils.metadata - INFO - 메타데이터 저장 완료: outputs/script/훈민정음_script.md.metadata.json


✅ 스크립트 생성 완료!

📄 결과 미리보기:
지금부터 여러분을 한글의 뿌리, 바로 훈민정음의 세계로 안내하겠습니다. 이름만 들어도 왠지 엄숙하고 신비로운 느낌이 들지 않으세요? 훈민정음은 조선 시대 세종대왕이 반포한, 오늘날 한글의 원형이자 한글의 탄생 비밀이 담긴 책입니다. 지금 우리말을 쉽게 읽고 쓸 수 있게 해준 주인공이죠.

이야기는 조선 전기, 그러니까 15세기 한양에서 시작됩니다. 당시 세종대왕은 백성들이 너무 어렵고 복잡한 한자 때문에 제대로 자신의 생각을 표현하지 못하는 상황을 안타깝게 생각했어요. 그래서 모두가 쉽고 편하게 쓸 수 있는 새로운 문자, 훈민정음을 만들기로 결심하게 됩니다. 이 때가 1443년, 그리고 공식적으로 온 나라에 반포된 해는 1446년입니다. 여러분, 혹시 세종대왕이 훈민정음을 만들게 된 진짜 이유가 뭘지 상상해보신 적 있으신가요? 바로 백성을 아끼는 따뜻한 마음이었습니다.

훈민정음의 진짜 대단한 점은 그 과학적인 원리에 있습니다. 자음은 사람의 발음 기관, 즉 입과 혀, 목구멍의 모양...
💾 결과 저장 완료: /Users/jacti/develop/script_gen_v2/test_results/script_generation/20251112_095024_v3-tts_훈민정음.md
✅ 저장 완료: 20251112_095024_v3-tts_훈민정음.md


## 📊 결과 비교 및 분석

In [ ]:
# 저장된 결과 파일 목록 조회

print("🔍 저장된 정보 검색 결과:")
print("=" * 80)
info_results = list_saved_results("info_retrieval")
for idx, path in enumerate(info_results[:10], 1):  # 최근 10개만 표시
    print(f"{idx}. {path.name}")

print("\n📝 저장된 스크립트 생성 결과:")
print("=" * 80)
script_results = list_saved_results("script_generation")
for idx, path in enumerate(script_results[:10], 1):  # 최근 10개만 표시
    print(f"{idx}. {path.name}")

In [ ]:
# 두 결과 파일 비교
# 위에서 출력된 파일 번호를 사용하여 비교

# === 설정 ===
PIPELINE_TO_COMPARE = "script_generation"  # "info_retrieval" 또는 "script_generation"
FILE1_INDEX = 0  # 비교할 첫 번째 파일 인덱스 (0부터 시작)
FILE2_INDEX = 1  # 비교할 두 번째 파일 인덱스

# === 실행 ===
results = list_saved_results(PIPELINE_TO_COMPARE)

if len(results) < 2:
    print("⚠️ 비교할 파일이 2개 미만입니다. 먼저 여러 프롬프트를 테스트하고 결과를 저장하세요.")
elif FILE1_INDEX >= len(results) or FILE2_INDEX >= len(results):
    print(f"❌ 인덱스 오류: 파일 인덱스는 0부터 {len(results)-1}까지 가능합니다.")
else:
    file1 = results[FILE1_INDEX]
    file2 = results[FILE2_INDEX]
    
    compare_results(file1, file2)

## 💡 팁

### 프롬프트 최적화 워크플로우

1. **베이스라인 설정**
   - 현재 default 프롬프트로 테스트 실행
   - 결과 저장 (baseline으로 활용)

2. **프롬프트 편집**
   - YAML 파일 직접 편집 또는 새 버전 생성
   - 변경 사항: 톤, 길이, 구조, 스타일 등

3. **A/B 테스트**
   - 동일한 키워드로 여러 프롬프트 버전 테스트
   - 각 결과 저장

4. **비교 분석**
   - 결과 비교 셀 사용
   - 품질, 길이, 톤 등 평가

5. **최종 선택**
   - 최적의 프롬프트 버전을 default로 설정

### 주의사항
- Dry-run 모드로 먼저 테스트하여 API 비용 절약
- 동일한 키워드로 비교해야 공정한 평가 가능
- Temperature 값 변경 시 결과가 달라질 수 있음